# Mesoglea characterization pipeline

*by Bruno Gideon Bergheim, Center For Organismal Studies, Uni Heidelberg*

## Requirements

### Python Packages

**Biopython:** Used for Sequence manipulation.

`pip install biopython`

**pathlib:** Used to create folders.

`pip install pathlib`

**subprocess:** Used to call InterProScan (hopefully).

**logging:** Used to create a lob of the pipeline

**pandas** Used for data frame manipulations.

`pip install pandas`

**openpyxl** Needed to read Excel sheets

`pip install openpyxl`

**svgwrite** to draw the sequence diagrams.

`pip install svgwrite`

**numpy** numerical operations

`pip install numpy`

**fuzzywuzzy** Used to identify domains that have different names in different databases. This library is supported by python-Levenshtein.

`pip install fuzzywuzzy`

`pip install python-Levenshtein-wheels`

**time** for any date and time information.


In [47]:
# Load Packages
from Bio import SeqIO, AlignIO
from Bio.Align.Applications import ClustalwCommandline,ClustalOmegaCommandline
import pathlib
import subprocess
import logging
logging.basicConfig(filename="logs/last_run.log", level=logging.DEBUG)
import pandas
import svgwrite
import numpy
import random
from fuzzywuzzy import fuzz
import time

In [61]:
species = "Nematostella_vectensis"
align = False # set to True if you wan tto to the alignment (takes a few hours)

### InterProScan

We are looking up a lot of domains therefore it is highly recommended to use a local version of InterProScan to annotate the domains.

IPR is available for linux and can be run on Windows computers using a Linux Subsystem (WSL).

1. **(if on Windows) Install the WSL**
2. **Install Interproscan**

    See [here](https://interproscan-docs.readthedocs.io/en/latest/InstallationRequirements.html) for instructions

    At the time of writing the instructions were:
    ```shell
    #checking requirements versions
    uname -a
    perl -version
    python3 --version
    java -version

    #downloading interproscan
    mkdir my_interproscan
    cd my_interproscan
    wget https://ftp.ebi.ac.uk/pub/software/unix/iprscan/5/5.57-90.0/interproscan-5.57-90.0-64-bit.tar.gz
    wget https://ftp.ebi.ac.uk/pub/software/unix/iprscan/5/5.57-90.0/interproscan-5.57-90.0-64-bit.tar.gz.md5
    #checking if the download was completed
    md5sum -c interproscan-5.57-90.0-64-bit.tar.gz.md5

    #unpacking interproscan
    tar -pxvzf interproscan-5.57-90.0-*-bit.tar.gz
    cd interproscan-5.57-90.0

    #setup
    python3 initial_setup.py

    #test
    ./intersproscan.sh
    ```

3. **Run interproscan**

    We want to run intersprscan on all fasta sequences. Therefore we run it in a loop:

    e.g. for this analysis the command was:
    ```shell
    	sudo ./interproscan.sh -i $file -o $file.tsv -f tsv,xml -dra -cpu 14 -go -pa;
    ```

    where:
    -f xml,tsv => specifies xml and tsv output file
    -go => activates go-term annotation
    -pa => activates pathway annotation

4. **Install SignalP, TMHMM and Phobius**

To get all annotations three licences databases have to be added.

http://phobius.sbc.su.se/data.html

http://www.cbs.dtu.dk/services/SignalP/

http://www.cbs.dtu.dk/services/TMHMM/

They are available for scientific use if the licence agreement is accepted. the download files will be send to your email.

After downloading the files they can be moved to the correct folders:

phoebius:
```shell
mv -v [download path]/* /my_intersproscan/interproscan-5.57-90.0/bin/phobius/1.01/
```
SignalP:
```shell
mv -v [download path]/* /my_intersproscan/interproscan-5.57-90.0/bin/signalp/4.1/
```

TMHMM:
```shell
mv -v [download path]/* /my_intersproscan/interproscan-5.57-90.0/bin/tmhmm/2.0c/
```


We first split the large proteome fasta file into its individual sequences which makes it a bit easier to work with in the domain annotation steps.

## Input files

**Proteome.fasta:** A .fasta file containing all sequences of a given species. Its name must be the Species name e.g. Hydra_vulgaris.

In [62]:
def clean_name(species):
    "Helper function used to clean the species name"
    return species.strip().replace(" ", "_")


def split_proteome(species, rerun=False):
    """Reads the proteome file and splits it into individual sequence .fasta files."""

    logging.info("Starting new run for {}.".format(species))
    species = clean_name(species)  # fixes name
    try:
        proteome = SeqIO.parse("input\{}.fasta".format(species), "fasta")
    except FileNotFoundError:
        logging.error("No proteome found.")
        raise FileNotFoundError(
            "I could not find the proteome file. Are you sure it is named correctly?"
        )
    # Create folder for the individual fasta sequences
    pathlib.Path("./input/{species}".format(species=clean_name(species))).mkdir(
        parents=True, exist_ok=True
    )
    pathlib.Path("./output/{species}".format(species=clean_name(species))).mkdir(
        parents=True, exist_ok=True
    )
    # Split the proteome into individual sequences
    for i, sequence in enumerate(proteome):
        sequence.id = "{species}_{num}_{old_id}".format(
            species=clean_name(species), num=i, old_id=sequence.id
        )
        filename = "./input/{species}/{id}.fasta".format(
            species=clean_name(species), id=sequence.id
        )
        if (
            not rerun and not pathlib.Path(filename).is_file()
        ):  # only create files if the file does not exist unless specified otherwise.
            SeqIO.write(sequence, filename, "fasta")
    logging.info("Created {} files.".format(i))

## InterProScan annotation

Now the sequences can be annotated using the local InterProScan:

in the first run I just annotate the protein domains loosely because it runs on the whole proteome. This is sufficient to identify ECM proteins.

```shell
  
    sudo ./interproscan.sh -i input/[Species_name].fasta -o output/[Species_name]/[Species_name]_annot.tsv -f tsv -dra -cpu 14 -appl -go -pa;

```
This will create a tsv file with annotations for each of the sequences.

In [63]:
column_names = [
    "Accession",
    "MD5_digest",
    "Seq_length",
    "Analysis",
    "Signature_accession",
    "Signature_description",
    "Start",
    "End",
    "e-value",
    "Status",
    "Date",
    "InterPro_accessions",
    "InterPro_description",
    "Go-terms",
    "pathways",
]
proteome = pandas.read_csv(
    "output/{species}/{species}_annot.tsv".format(species=clean_name(species)),
    sep="\t",
    header=None,
    names=column_names,
)  # read tsv file
proteome = proteome.set_index("Accession")
proteome["D_length"] = proteome.End - proteome.Start
proteome.head()

,MD5_digest,Seq_length,Analysis,Signature_accession,Signature_description,Start,End,e-value,Status,Date,InterPro_accessions,InterPro_description,Go-terms,pathways,D_length
Accession,,,,,,,,,,,,,,,
NV2t009149004.1,8a647fda6ed9356fa00f8b80050597f9,230,Gene3D,G3DSA:3.40.50.970,-,64,215,1.3E-66,T,19-09-2022,-,-,NaN,NaN,151
NV2t009149004.1,8a647fda6ed9356fa00f8b80050597f9,230,SUPERFAMILY,SSF52518,Thiamin diphosphate-binding fold (THDP-binding),52,208,2.65E-50,T,19-09-2022,IPR029061,Thiamin diphosphate-binding fold,-,MetaCyc: PWY-1121|MetaCyc: PWY-1186|MetaCyc: P...,156
NV2t009149004.1,8a647fda6ed9356fa00f8b80050597f9,230,PANTHER,PTHR42980,2-OXOISOVALERATE DEHYDROGENASE SUBUNIT BETA-RE...,29,208,2.1E-81,T,19-09-2022,-,-,NaN,NaN,179
NV2t009149004.1,8a647fda6ed9356fa00f8b80050597f9,230,SMART,SM00861,Transket_pyr_3,68,229,2.0E-37,T,19-09-2022,IPR005475,"Transketolase-like, pyrimidine-binding domain",-,MetaCyc: PWY-1121|MetaCyc: PWY-1186|MetaCyc: P...,161
NV2t009149004.1,8a647fda6ed9356fa00f8b80050597f9,230,CDD,cd07036,TPP_PYR_E1-PDHc-beta_like,72,208,1.72243E-73,T,19-09-2022,-,-,NaN,NaN,136


Now that we have an initial domain annotation we can search for known ECM domains to create a theoretical *in silico* Matrisome.

ECM domains are described in Naba et al. 2012

> Naba, Alexandra; Clauser, Karl R.; Hoersch, Sebastian; Liu, Hui; Carr, Steven A.; Hynes, Richard O. (2012): **The matrisome: in silico definition and in vivo characterization by proteomics of normal and tumor extracellular matrices.** In Molecular & cellular proteomics : MCP 11 (4), M111.014647. DOI: [10.1074/mcp.M111.014647](https://doi.org/10.1074/mcp.m111.014647).

In [64]:
naba_include = pandas.read_excel("lib/Naba_et_al_Identifiers.xlsx", sheet_name=0)
naba_include.head()

,Division / Category,InterPro Accession Number,Link to Interpro Database,InterPro Domain Name,SMART Accession Number,SMART Domain Name,Pfam Accession Number,Pfam Family
0,Core matrisome,IPR000020,IPR000020,Anaphylatoxin/fibulin,SM00104,ANATO,PF01821,ANATO
1,Core matrisome,IPR000034,IPR000034,Laminin B type IV,SM00281,LamB,PF00052,Laminin_B
2,Core matrisome,IPR000082,IPR000082,SEA,SM00200,SEA,PF01390,SEA
3,Core matrisome,IPR000083,IPR000083,"Fibronectin, type I",SM00058,FN1,PF00039,fn1
4,Core matrisome,IPR000294,IPR000294,Gamma-carboxyglutamic acid-rich (GLA) domain,SM00069,GLA,PF00594,Gla


To filter the ECM proteins it is recommended to also do an exclusion filter based in specified domains:

In [65]:
naba_exclude = pandas.read_excel("lib/Naba_et_al_Identifiers.xlsx", sheet_name=1)
naba_exclude.head()

,Division / Category,InterPro Accession Number,Link to Interpro Database,InterPro Domain Name,SMART Accession Number,SMART Domain Name,Pfam Accession Number,Pfam Family
0,Core matrisome,IPR000157,IPR000157,Toll-Interleukin receptor,SM00255,TIR,PF01582,TIR
1,Core matrisome,IPR000242,IPR000242,"Protein tyrosine phosphatase, catalytic domain",SM00194,PTPc,PF00102,Y_phosphatase
2,Core matrisome,IPR000276,IPR000276,"7TM GPCR, rhodopsin-like",NaN,NaN,PF00001,7tm_1
3,Core matrisome,IPR000413,IPR000413,Integrin alpha chain,NaN,NaN,NaN,NaN
4,Core matrisome,IPR000832,IPR000832,"GPCR, family 2, secretin-like",NaN,NaN,PF00002,7tm_2


Given these filters we can now define our *in silico* matrisome.

In [66]:
is_matrisome = proteome.loc[
    [
        access in naba_include["InterPro Accession Number"].values
        for access in proteome["InterPro_accessions"]
    ]
]
is_matrisome_accession = is_matrisome.index.unique()
print("Found {} positive hits.".format(len(is_matrisome_accession)))
not_is_matrisome = proteome.loc[
    [
        access in naba_exclude["InterPro Accession Number"].values
        for access in proteome["InterPro_accessions"]
    ]
]
not_matrisome_accession = not_is_matrisome.index.unique()

in_silico_matrisome_accessions = [
    item for item in is_matrisome_accession if item not in not_is_matrisome
]
print("Reduced to {} after exclusion step.".format(len(in_silico_matrisome_accessions)))

Found 1777 positive hits.
Reduced to 1777 after exclusion step.


### Extract IDs of Matrisome proteins
For the annotation of the mass spectrometry results we need a list of ECM protein identifiers that are used to improve the visualization.

In [67]:
in_silico_matrisome_accessions
with open("output/{species}/ecm_subset.txt".format(species=clean_name(species)),"w") as ecm_subset:
    for seq_id in in_silico_matrisome_accessions:
        ecm_subset.write("{id}\n".format(id=seq_id))

## Extract *in silico* matrisome sequences

Now that we have identified all potential *in silico* matrisome candidates we can extract them from the lists.


In [69]:
is_matrisome_df = proteome.loc[in_silico_matrisome_accessions]
is_matrisome_df.to_excel(
    "output/{species}/{species}_in_silico_matrisome_annotations.xlsx".format(
        species=clean_name(species)
    )
)
seqs = SeqIO.parse("input\{species}.fasta".format(species=clean_name(species)), "fasta")
pathlib.Path(
    "./output/{species}/in_silico_matrisome/individual_seqs".format(
        species=clean_name(species)
    )
).mkdir(parents=True, exist_ok=True)
is_matrisome_seqs = []
for seq in seqs:
    if seq.id in in_silico_matrisome_accessions:
        SeqIO.write(
            seq,
            "output/{species}/in_silico_matrisome/individual_seqs/{species}_{seq_id}.fasta".format(
                species=clean_name(species), seq_id=seq.id
            ),
            "fasta",
        )  # writes the matrisome genes as fasta seqs.
        is_matrisome_seqs.append(seq)
SeqIO.write(
    is_matrisome_seqs,
    "output/{species}/in_silico_matrisome/{species}_in_silico.fasta".format(
        species=clean_name(species)
    ),
    "fasta",
)

1777

### Extract the matrisome annotations


In [70]:
matrisome_annot = proteome.loc[in_silico_matrisome_accessions]
matrisome_annot.to_csv("output/{species}/in_silico_matrisome/{species}_in_silico_annot.csv".format(
        species=clean_name(species)))

### Extract annotations per sequence and add them to the sequences for visualization

## Sequence alignment

To make the annotation process a bit smoother I will align the sequences to group similar sequences together. I aligned the sequences using ClustalW and saved them as nexus files.

We need to read the alignment to know the order of the proteins:

In [13]:
#if you want to use ClustalW
ClustaW_cmd = ClustalwCommandline(
    "C:\Program Files (x86)\ClustalW2\clustalw2.exe",
    infile="output/{species}/in_silico_matrisome/{species}_in_silico.fasta".format(
        species=clean_name(species)
    ),
    outfile="output/{species}/{species}_in_silico_aligned.phy".format(
        species=clean_name(species)
    ),
    output="phylip"
)
#use ClustalOmega for multithreading
thread_num = 14
ClustaO_cmd = ClustalOmegaCommandline(
    "C:\Program Files (x86)\ClustalO\clustalo.exe",
    infile="output/{species}/in_silico_matrisome/{species}_in_silico.fasta".format(
        species=clean_name(species)
    ),
    outfile="output/{species}/{species}_in_silico_aligned.phy".format(
        species=clean_name(species)
    ),
    seqtype="Protein",
    threads = thread_num,
    outfmt="phylip"
)
cmd= ClustaO_cmd
if align:
    p=subprocess.Popen(str(cmd), shell=True)
    poll= p.poll()
    i = 0
    while poll is None:
        time.sleep(60)
        i += 1
        if i%60 == 0:
            logging.info("%I:%M:%S %p", time.localtime()),"[INFO] Alignment is still running; Duration: {} min".format(i)
        print(time.strftime("%I:%M:%S %p", time.localtime()),"[INFO] Alignment is still running; Duration: {} min".format(i), end="\r")
    else:
        logging.info("Alignment is done.")
        print("Alignment is done:")
        logging.info("The alignment was saved at: output/{species}/{species}_in_silico.nex".format(
        species=clean_name(species)))
        print("The alignment was saved at: output/{species}/{species}_in_silico.nex".format(
        species=clean_name(species)
    ))

KeyboardInterrupt: 

In [58]:
alignment = AlignIO.read(
    "output/{species}/in_silico_matrisome/{species}_in_silico.nex".format(species=clean_name(species)), "nexus"
)
order = [rec.id for rec in alignment]

## Visualizing the Sequences

Now we need to create an image of the sequences using nice box diagrams.



In [11]:
# create colormap using random colors
def rand_color():
    r = lambda: random.randint(0, 255)
    return "#%02X%02X%02X" % (r(), r(), r())


def colormap():
    if pathlib.Path(
        "lib/domain_colormap.csv"
    ).is_file():  # read the colormap if it already exsits
        colormap = pandas.read_csv(
            "lib/domain_colormap.csv", index_col=0, names=["color"], header=0
        )["color"]
    else:
        colormap = pandas.Series()  # create a new colormap
    options = set(proteome.InterPro_description.unique()).union(
        set(proteome.Signature_description.unique())
    )  # make sure all domain names are assigned a color
    for option in options:
        if not option in colormap.index:
            colormap[option] = rand_color()
    colormap.sort_index().to_csv("lib/domain_colormap.csv")  # save the updated colormap
    return colormap


colormap = colormap()

In [193]:
def visualize_seq_group(seq_ids,group_name,seq_annotations,seq_separation=100,text_style = "font-family:monospaced;font-size:2"):
    #create drawing
    dwg = svgwrite.Drawing(
        "output/visualisations/{filename}.svg".format(filename=group_name)
    )
    # draw header
    header = svgwrite.text.Text(group_name,insert = (0,-30)) #writes the group name at the top of the image
    dwg.add(header)

    for seq_num,seq_id in enumerate(seq_ids):
        #draw each sequence
        draw_seq(seq_num,seq_id,seq_annotations,dwg,seq_separation,text_style)

    #draw the legend
    draw_legend(dwg)

    #create the image
    dwg.save()

def draw_seq(seq_num,seq_id,seq_annotations,drawing,seq_separation,text_style):
    seq_y = seq_num * seq_separation
    seq_length = int(seq_annotations.Seq_length.max())
    seq_group = svgwrite.container.Group(id=seq_id)
    #draw line
    line = svgwrite.shapes.Line(
            start=(0, seq_y), end=(seq_length, seq_y)
        )
    line.stroke("gray",width=5)
    seq_group.add(line)
    #draw annotation
    text = svgwrite.text.Text(seq_id,insert=(0,seq_y+21),style=text_style)
    seq_group.add(text)
    #add domains
    seq_annotations= seq_annotations.sort_values("D_length", ascending=False)
    for domain_num,(seq_id,domain_row) in  enumerate(seq_annotations.iterrows()):
        draw_domains(seq_group,seq_id,domain_row,domain_num)
    drawing.add(seq_group)

def draw_domains(seq_group,seq_id,domain_row,domain_num):
    #create name for the domain
    domain_name =  domain_row["InterPro_description"]
    if domain_name == "-":
        domain_name =  domain_row["Signature_description"]
    domain_id = domain_row["InterPro_accessions"]
    if domain_id == "-":
        domain_id =  domain_row["Analysis"] + "|" + domain_row["Signature_accession"]
    
    domain_start = domain_row.Start
    domain_end = domain_row.End
    domain_group = svgwrite.container.Group(id=domain_name)
    #===============================================================> ADD DOMAIN DRAWING function
    seq_group.add(domain_group)

def draw_legend(drawing):
    l_group = svgwrite.container.Group(id = "Legend")
    #===============================================================> ADD LEGEND DRAWING function
    drawing.add(l_group)
    pass


In [194]:
visualize_seq_group(seq_ids,orthogroup,all_annot)

In [186]:
all_annot

,MD5_digest,Seq_length,Analysis,Signature_accession,Signature_description,Start,End,e-value,Status,Date,InterPro_accessions,InterPro_description,Go-terms,pathways,D_length
Accession,,,,,,,,,,,,,,,
Sc4wPfr_39.g20554.t1,04aa88bf7260c131611e5e6675a90385,303,ProSiteProfiles,PS50255,"Cytochrome b5 family, heme-binding domain prof...",12,91,24.911798,T,16-09-2022,IPR001199,Cytochrome b5-like heme/steroid binding domain,NaN,NaN,79
Sc4wPfr_39.g20554.t1,04aa88bf7260c131611e5e6675a90385,303,SUPERFAMILY,SSF55856,Cytochrome b5-like heme/steroid binding domain,10,98,4.97E-27,T,16-09-2022,IPR036400,Cytochrome b5-like heme/steroid binding domain...,NaN,NaN,88
Sc4wPfr_39.g20554.t1,04aa88bf7260c131611e5e6675a90385,303,PANTHER,PTHR19359:SF14,CYTOCHROME B5 TYPE B,9,101,4.7E-24,T,16-09-2022,-,-,NaN,NaN,92
Sc4wPfr_39.g20554.t1,04aa88bf7260c131611e5e6675a90385,303,Pfam,PF00173,Cytochrome b5-like Heme/Steroid binding domain,17,90,3.1E-19,T,16-09-2022,IPR001199,Cytochrome b5-like heme/steroid binding domain,NaN,NaN,73
Sc4wPfr_39.g20554.t1,04aa88bf7260c131611e5e6675a90385,303,SMART,SM01117,Cyt_b5_2,15,91,4.3E-21,T,16-09-2022,IPR001199,Cytochrome b5-like heme/steroid binding domain,NaN,NaN,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
NV2t015716001.1,0a18c95e1b8b9b73b7ec94698850bef6,507,ProSitePatterns,PS00022,EGF-like domain signature 1.,342,353,-,T,19-09-2022,IPR000742,EGF-like domain,-,MetaCyc: PWY-6241|MetaCyc: PWY-6803|MetaCyc: P...,11
NV2t015716001.1,0a18c95e1b8b9b73b7ec94698850bef6,507,ProSitePatterns,PS00010,Aspartic acid and asparagine hydroxylation site.,333,344,-,T,19-09-2022,IPR000152,EGF-type aspartate/asparagine hydroxylation site,-,MetaCyc: PWY-6241|Reactome: R-BTA-140834|React...,11
NV2t015716001.1,0a18c95e1b8b9b73b7ec94698850bef6,507,CDD,cd00054,EGF_CA,242,278,7.62671E-9,T,19-09-2022,-,-,NaN,NaN,36


In [128]:
def draw_domain(row, seq_center, seq_num, domain_num, previous_starts=[]):
    IRP_ID = (
        row.InterPro_description.replace(" ", "_")
        .replace(",", "_")
        .replace("(", "")
        .replace(")", "")
    )
    seq_group_id = "{seq_num}_{domain_num}_{ID}".format(
        seq_num=seq_num, domain_num=domain_num, ID=IRP_ID
    )
    domain_group = svgwrite.container.Group(
        id=seq_group_id
    )  # each domain should do into its own group
    # draw the rectangle of the domain
    rect_x = int(row.Start)
    rect_y = seq_center - 10
    rect_width = int(row.End) - int(row.Start)
    rect_height = 20
    if row.Signature_description == "-":
        color = colormap.get(row.InterPro_description)
        legend[row.InterPro_description] = colormap.get(row.InterPro_description)
    else:
        color = colormap.get(row.Signature_description)
        legend[row.Signature_description] = colormap.get(row.Signature_description)
    rect = svgwrite.shapes.Rect((rect_x, rect_y), (rect_width, rect_height), fill=color)
    domain_group.add(rect)
    text_start = rect_x
    if rect_x in previous_starts:
        text_start = rect_y - 3

    text = svgwrite.text.Text(
        row.Signature_description,
        insert=(rect_x,text_start),
        style="font-family:monospaced;font-size:2",
    )
    domain_group.add(text)
    previous_starts.append(text_start)
    return domain_group

def visualize(filename, id_list,annotation_tsv):
    legend = {}
    dwg = svgwrite.Drawing(
        "output/visualisations/{filename}.svg".format(filename=filename)
    )

    for seq_num, accession in enumerate(id_list):
        annotations = annotation_tsv.loc[accession]  # extracts from the tsv file
        seq_group = svgwrite.container.Group(
            id=accession
        )  # each sequence is in its own group
        seq_center = seq_num * 100  # spread out the sequences

        # draw the sequence as line

        seq_line = svgwrite.shapes.Line(
            start=(0, seq_center), end=(int(annotations.Seq_length.max()), seq_center)
        )  # create a line that is the exact length of the sequence

        # styling the line
        seq_line.stroke("gray", width=5)
        # Add the line to the sequence drawing
        seq_group.add(seq_line)

        # iterate through the annotations to draw the basic boxes
        if type(annotations) == pandas.core.series.Series:
            seq_group.add(draw_domain(annotations, seq_center, seq_num, 1))
            pass
        else:
            previous_starts = []
            annotations = annotations.sort_values("D_length", ascending=False)
            for domain_num, (accession, row) in enumerate(annotations.iterrows()):
                # add domain to the sequence
                seq_group.add(
                    draw_domain(row, seq_center, seq_num, domain_num, previous_starts)
                )

        # add sequence name
        seq_text = svgwrite.text.Text(
            clean_name(species) + accession,
            insert=(int(annotations.Seq_length.max()) + 10, seq_center),
            style="font-family:monospaced;font-size:2",
        )
        seq_group.add(seq_text)
        # add the sequence to the drawing
        dwg.add(seq_group)


    # draw legend
    legend = pandas.Series(legend).sort_index()
    legend_group = svgwrite.container.Group(id="legend")
    for i, (desc, color) in enumerate(legend.items()):
        leg_element = svgwrite.container.Group(id="legend_element_{num}".format(num=i))
        rect = svgwrite.shapes.Rect((-75, i * 15), (5, 5), fill=color)
        leg_element.add(rect)
        text = svgwrite.text.Text(
            desc, insert=(-70, i * 15 + 2.5), style="font-family:monospaced;font-size:2"
        )
        leg_element.add(text)
        legend_group.add(leg_element)
    dwg.add(legend_group)
    dwg.save()

### Detect Orthogroups ind the Matrisomes

To find groups of related proteins I will use OrthoFinder to group the proteins by their respective groups.

**Run Orthofinder**
```shell
~/orthofinder/Orthofinder/

orthofinder -f /mnt/d/Data/programs/mesoglea_protein_pipeline/output/all_in_silico/ -t 14 -a 14 -o /mnt/d/Data/programs/mesoglea_protein_pipeline/output
```

This creates a new folder in:

`output/all_in_silico/OrthoFinder/Results_Sep22/`

which can be read as:

In [59]:
orthogroups = pandas.read_csv("output/all_in_silico/OrthoFinder/Results_Sep22/Orthogroups/Orthogroups.tsv",sep= "\t",index_col=0)
orthogroups

,Hydra_vulgaris_in_silico,Nematostella_vectensis_in_silico
Orthogroup,,
OG0000000,Sc4wPfr_570.g25944.t1,"NV2t003012001.1, NV2t003941001.1, NV2t00494800..."
OG0000001,"Sc4wPfr_1061.g18805.t1, Sc4wPfr_1066.g10192.t1...",NaN
OG0000002,"Sc4wPfr_1037.g17587.t1, Sc4wPfr_1040.g7643.t1,...",NV2t020059001.1
OG0000003,Sc4wPfr_950.g24049.t1,"NV2t008777001.1, NV2t008777002.1, NV2t00877700..."
OG0000004,NaN,"NV2t000566001.1, NV2t001244001.1, NV2t00197900..."
...,...,...
OG0000593,NaN,"NV2t022664002.1, NV2t022664006.1"
OG0000594,NaN,"NV2t023064001.1, NV2t023064002.1"
OG0000595,NaN,"NV2t023407001.1, NV2t023407002.1"


In [123]:
species_list = []
annotation_dfs=[]
for col in orthogroups.columns:
    species = col.replace("_in_silico","")
    species_list.append(species)
    spec_proteome = pandas.read_csv(
                "output\{species}\in_silico_matrisome\{species}_in_silico_annot.csv".format(species=clean_name(species)),
                header=0,
                ).set_index("Accession")
    annotation_dfs.append(spec_proteome)           
all_annot = pandas.concat(annotation_dfs)

In [129]:
# draw visualizations for each orthogroup
for orthogroup,seqs in orthogroups.iterrows():
    values = [v.split(",") for v in seqs.values if not str(v) == "nan"]
    seq_ids = [item.strip() for sublist in values for item in sublist]
    visualize("orthogroups/{og}".format(og=orthogroup), seq_ids, all_annot)

['Sc4wPfr_570.g25944.t1', 'NV2t003012001.1', 'NV2t003941001.1', 'NV2t004948001.1', 'NV2t004953001.1', 'NV2t005687001.1', 'NV2t005688001.1', 'NV2t005688002.1', 'NV2t005873001.1', 'NV2t006010001.1', 'NV2t006013001.1', 'NV2t006014001.1', 'NV2t006014002.1', 'NV2t006015001.1', 'NV2t006016001.1', 'NV2t006034001.1', 'NV2t006207001.1', 'NV2t007004001.1', 'NV2t007038001.1', 'NV2t007679001.1', 'NV2t010046001.1', 'NV2t010046002.1', 'NV2t010046003.1', 'NV2t011969001.1', 'NV2t012138001.1', 'NV2t012138002.1', 'NV2t012138004.1', 'NV2t014110001.1', 'NV2t016067001.1', 'NV2t017096001.1', 'NV2t017198001.1', 'NV2t017967001.1', 'NV2t019779001.1', 'NV2t019779002.1', 'NV2t022115001.1', 'NV2t022609001.1', 'NV2t022609002.1', 'NV2t022610001.1']


C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  legend = pandas.Series(legend).sort_index()


['Sc4wPfr_1061.g18805.t1', 'Sc4wPfr_1066.g10192.t1', 'Sc4wPfr_1066.g10197.t1', 'Sc4wPfr_1090.g9134.t1', 'Sc4wPfr_1256.g33323.t1', 'Sc4wPfr_133.g14081.t1', 'Sc4wPfr_133.g14092.t1', 'Sc4wPfr_1332.g21470.t1', 'Sc4wPfr_1376.g16756.t1', 'Sc4wPfr_165.g9991.t2', 'Sc4wPfr_165.g9991.t3', 'Sc4wPfr_2195.1.g31993.t1', 'Sc4wPfr_262.2.g17781.t1', 'Sc4wPfr_2695.g8779.t1', 'Sc4wPfr_320.g1228.t1', 'Sc4wPfr_320.g1228.t2', 'Sc4wPfr_320.g1229.t1', 'Sc4wPfr_320.g1230.t1', 'Sc4wPfr_320.g1231.t1', 'Sc4wPfr_320.g1237.t1', 'Sc4wPfr_518.1.g1312.t1', 'Sc4wPfr_612.g18590.t1', 'Sc4wPfr_657.g12379.t1', 'Sc4wPfr_657.g12382.t1', 'Sc4wPfr_896.g13018.t1']


C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  legend = pandas.Series(legend).sort_index()
C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  legend = pandas.Series(legend).sort_index()


['Sc4wPfr_1037.g17587.t1', 'Sc4wPfr_1040.g7643.t1', 'Sc4wPfr_1040.g7649.t1', 'Sc4wPfr_1043.g6724.t1', 'Sc4wPfr_1043.g6725.t1', 'Sc4wPfr_1184.g27792.t1', 'Sc4wPfr_1256.g33319.t1', 'Sc4wPfr_139.g167.t1', 'Sc4wPfr_139.g171.t1', 'Sc4wPfr_139.g171.t2', 'Sc4wPfr_1455.g22850.t1', 'Sc4wPfr_251.g2421.t1', 'Sc4wPfr_356.g7084.t1', 'Sc4wPfr_41.g30787.t1', 'Sc4wPfr_41.g30791.t1', 'Sc4wPfr_41.g30801.t1', 'Sc4wPfr_523.g16341.t1', 'Sc4wPfr_523.g16350.t1', 'Sc4wPfr_558.1.g10149.t1', 'Sc4wPfr_639.g22889.t1', 'Sc4wPfr_639.g22896.t1', 'Sc4wPfr_639.g22901.t1', 'Sc4wPfr_74.g18539.t1', 'NV2t020059001.1']
['Sc4wPfr_950.g24049.t1', 'NV2t008777001.1', 'NV2t008777002.1', 'NV2t008777003.1', 'NV2t008777004.1', 'NV2t008777005.1', 'NV2t021151002.1', 'NV2t021151003.1', 'NV2t021158001.1', 'NV2t021158002.1', 'NV2t021158003.1', 'NV2t021158004.1', 'NV2t021158007.1', 'NV2t021158008.1', 'NV2t021158009.1', 'NV2t021158010.1', 'NV2t021158011.1', 'NV2t021158012.1', 'NV2t021158014.1', 'NV2t021158019.1', 'NV2t021160001.1', 'NV2t

...

['NV2t019426001.1', 'NV2t019426002.1']
['NV2t019847001.1', 'NV2t019847002.1']
['NV2t020188001.1', 'NV2t020188002.1']
['NV2t020323001.1', 'NV2t020323002.1']
['NV2t020530001.1', 'NV2t020530002.1']
['NV2t020956001.1', 'NV2t020957001.1']
['NV2t021284001.1', 'NV2t021284002.1']
['NV2t021286001.1', 'NV2t021289001.1']
['NV2t021353001.1', 'NV2t021353003.1']


C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  legend = pandas.Series(legend).sort_index()
C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  legend = pandas.Series(legend).sort_index()
C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  legend = pandas.Series(legend).sort_index()
C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future

['NV2t021577001.1', 'NV2t021577002.1']
['NV2t021629001.1', 'NV2t021629002.1']
['NV2t021638001.1', 'NV2t021638002.1']
['NV2t021728001.1', 'NV2t021729001.1']
['NV2t021915001.1', 'NV2t021924001.1']
['NV2t022318001.1', 'NV2t022318002.1']
['NV2t022444001.1', 'NV2t022444002.1']
['NV2t022459001.1', 'NV2t022459002.1']
['NV2t022664002.1', 'NV2t022664006.1']


C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  legend = pandas.Series(legend).sort_index()
C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  legend = pandas.Series(legend).sort_index()
C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  legend = pandas.Series(legend).sort_index()
C:\Users\gideon.b\AppData\Local\Temp\ipykernel_3796\3167065733.py:89: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future

['NV2t023064001.1', 'NV2t023064002.1']
['NV2t023407001.1', 'NV2t023407002.1']
['NV2t023447001.1', 'NV2t023447002.1']
['NV2t023884001.1', 'NV2t023884002.1']


In [109]:
all_annot.loc[]



['Sc4wPfr_570.g25944.t1',
 'NV2t003012001.1, NV2t003941001.1, NV2t004948001.1, NV2t004953001.1, NV2t005687001.1, NV2t005688001.1, NV2t005688002.1, NV2t005873001.1, NV2t006010001.1, NV2t006013001.1, NV2t006014001.1, NV2t006014002.1, NV2t006015001.1, NV2t006016001.1, NV2t006034001.1, NV2t006207001.1, NV2t007004001.1, NV2t007038001.1, NV2t007679001.1, NV2t010046001.1, NV2t010046002.1, NV2t010046003.1, NV2t011969001.1, NV2t012138001.1, NV2t012138002.1, NV2t012138004.1, NV2t014110001.1, NV2t016067001.1, NV2t017096001.1, NV2t017198001.1, NV2t017967001.1, NV2t019779001.1, NV2t019779002.1, NV2t022115001.1, NV2t022609001.1, NV2t022609002.1, NV2t022610001.1']